In [1]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch
from langchain_community.document_loaders import PyPDFLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import UnstructuredExcelLoader


from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

d:\jincheng_project\RAG\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained(
#     r"D:\Backend_insurance\Algorithm\Fine_tuning\DeepSeek-R1-Distill-Llama-8B", 
#     trust_remote_code=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     r"D:\Backend_insurance\Algorithm\Fine_tuning\DeepSeek-R1-Distill-Llama-8B", 
#     trust_remote_code=True,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     load_in_4bit=True
# )


# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
# llm = HuggingFacePipeline(pipeline=pipe)

In [3]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

llm = OllamaLLM(model="deepseek-r1:8b", base_url="http://localhost:11434", temperature = 0.3)


In [5]:
loader = UnstructuredExcelLoader(r"D:\jincheng_project\RAG\pension_complaints_rewritten.xlsx", mode="elements")
do = loader.load()

all_splits  = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=200,
        add_start_index=True
    ).split_documents(do)

In [6]:
embeddings = HuggingFaceEmbeddings(model_name=r"D:\jincheng_project\RAG\all-mpnet-base-v2")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_40948\2404468741.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=r"D:\jincheng_project\RAG\all-mpnet-base-v2")


In [7]:
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [8]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [13]:
Template = """
你是一个专业的养老保险索赔助理.请根据过去的经验直接回答用户的询问。答案必须准确。请使用专业语言，不要使用第一人称。如果有人询问电话号码，请提供正确答案。如果您不知道电话号码，请告诉我们您没有关于该号码的信息。如果有人询问并且涉及相关部门的名称，并且您有关于该部门的信息，请提及该部门。如果你没有资料, 请避免提供错误的名称。

请记住，你是保险专家，如果有人问你的问题，这是完全不相关的，而不是试图根据保险上下文回答，你必须把它通用和一般的反应。

例如:
问：你好，你好吗 
答：我很好，谢谢。
这是通用的响应，你必须像我在示例中那样以通用的方式处理它

上下文环境：
{context}

问题：
{question}

回答：
"""

prompt = PromptTemplate.from_template(Template)

In [14]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=1)
    if not retrieved_docs:
        return {"context": [Document(page_content="无相关信息")]}
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    formatted_prompt = prompt.format(question=state["question"], context=docs_content)
    
    # Get just the generated text without the prompt
    response = llm(formatted_prompt)
    
    # Extract just the answer part (you may need to adjust this based on your model's output format)
    if isinstance(response, dict):
        answer = response.get("generated_text", "").replace(formatted_prompt, "").strip()
    else:
        answer = response.strip()
    
    return {"answer": answer}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [17]:
response = graph.invoke({"question": "你好， 朋友。 我 怎么了"})


In [19]:
print(response['answer'])

<think>
嗯，我现在需要帮助用户解决一个关于养老金领取的问题。根据上下文，用户提到他们在高平市领取养老金，并且是在2024年4月18日通过民生山西APP进行资格认证时遇到了系统维护的问题，无法完成认证。

首先，我应该确认用户的具体情况：他们是高平市的退休居民，使用了民生山西APP进行认证，但系统出现了问题。接下来，我需要提供解决方案，帮助用户处理这个问题。

根据之前的经验，当遇到系统维护或无法完成在线认证时，最好的建议通常是耐心等待维护结束后再次尝试，或者通过其他渠道进行认证，比如电话联系相关部门或走访当地服务中心。这样可以确保用户能够顺利完成资格认证，并及时领取养老金。

另外，我还应该提醒用户，如果在等待期间有任何问题，可以再次拨打客服热线进行咨询，这样他们可以获得更具体的指导和帮助。

总结一下，我的回答应该包括以下几点：
1. 建议用户耐心等待系统维护结束后，再次尝试通过民生山西APP进行认证。
2. 提供其他可选的认证渠道，如电话联系相关部门或前往当地服务中心。
3. 提醒用户如果仍有问题，可以拨打客服热线进一步咨询。

这样既解决了当前的问题，又提供了多种解决方案，确保用户能够顺利完成养老金领取的过程。
</think>

您好。针对您在高平市领取养老金的情况，您可以尝试通过以下方式处理：

1. **耐心等待**：系统维护结束后，请再次尝试通过民生山西APP进行认证。

2. **多渠道认证**：如仍有问题，可考虑通过电话联系相关部门或前往当地养老服务中心进行走访认证。

3. **进一步咨询**：如果仍有疑问，欢迎拨打相关部门的客服热线进行详细咨询和指导。

建议您耐心等待并多渠道尝试，以便顺利完成资格认证。如有需要，可再次来电咨询以获取更具体信息。


In [1]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch
from langchain_community.document_loaders import PyPDFLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import UnstructuredExcelLoader


from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_ollama.llms import OllamaLLM

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

class InsuranceModels:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.load_models()
    
    def load_models(self):
        self.model = OllamaLLM(model="deepseek-r1:14b", base_url="http://localhost:11434")
        self.embeddings = self.text_embedding()
        self.vector_store = self.vector_search()
        self.prompt = self.format_prompt()
        self.all_splits = self.doc_splitting()
        _ = self.vector_store.add_documents(documents=self.all_splits)
        self.graph = self.load() 
    def doc_splitting(self):
        loader = UnstructuredExcelLoader(r"D:\jincheng_project\RAG\pension_complaints_rewritten.xlsx", mode="elements")
        do = loader.load()

        all_splits  = RecursiveCharacterTextSplitter(
                chunk_size=400,
                chunk_overlap=200,
                add_start_index=True
            ).split_documents(do)
        return all_splits
    def text_embedding(self):
        return HuggingFaceEmbeddings(model_name=r"D:\jincheng_project\RAG\all-mpnet-base-v2")
    def vector_search(self):
        embedding_dim = len(self.embeddings.embed_query("hello world"))
        index = faiss.IndexFlatL2(embedding_dim)

        vector_store = FAISS(
            embedding_function=self.embeddings,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )
        return vector_store
    
    def retrieve(self, state: State):
        retrieved_docs = self.vector_store.similarity_search(state["question"], k=1)
        if not retrieved_docs:
            return {"context": [Document(page_content="无相关信息")]}
        return {"context": retrieved_docs}

    def generate(self, state: State):
        docs_content = "\n\n".join(doc.page_content for doc in state["context"])
        formatted_prompt = self.prompt.format(question=state["question"], context=docs_content)
        
        # Get just the generated text without the prompt
        response = self.model(formatted_prompt)
        
        # Extract just the answer part (you may need to adjust this based on your model's output format)
        if isinstance(response, dict):
            answer = response.get("generated_text", "").replace(formatted_prompt, "").strip()
        else:
            answer = response.strip()
        
        
        return {"answer": answer}

    def format_prompt(self):
        Template = """
            您是专业的养老保险理赔助理。请根据以往经验直接回答用户的咨询。答案必须准确。请使用专业语言，请勿使用第一人称。如果有人询问电话号码，请提供正确答案。如果您不知道电话号码，请告知您没有该号码的信息。
            如果有人询问，并且涉及相关部门名称，并且您有该部门的信息，请提及该部门。如果您没有信息，请避免提供错误的名称。如果有人问一般的问题，你必须相应地回答

            上下文：
            {context}

            问题：
            {question}

            答案：
            """

        prompt = PromptTemplate.from_template(Template)
        return prompt
    def load(self):
        # Compile application and test
        graph_builder = StateGraph(State).add_sequence([self.retrieve, self.generate])
        graph_builder.add_edge(START, "retrieve")
        graph = graph_builder.compile()
        return graph

    def transcribe(self, question: str):
        response = self.graph.invoke({"question": question})    
        return response     

d:\jincheng_project\RAG\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ins = InsuranceModels()



In [5]:
res = ins.transcribe("没有人告诉我逾期保险费，现在我有罚款。我做什么")
rr = str(res['answer']).split("</think>")[-1]
print(rr)



关于您提到的养老保险费逾期导致罚款的问题，通常情况下，参保人员需要按照社会保险法规定的时间节点缴纳保险费用。如果您因故未能按时缴纳，可能会产生滞纳金。建议您尽快联系当地的社会保险经办机构或通过官方提供的线上平台查询您的账户状态，并了解具体的补缴流程和相关政策规定。
